# 05 - RSA: математика и реализация

Этот notebook покрывает урок CRYPTO-08 (RSA).

**Что вы изучите:**
1. Учебная реализация RSA с маленькими простыми числами
2. Шифрование и дешифрование с RSA
3. Продакшен RSA с pycryptodome (RSA + OAEP)
4. Сравнение размеров ключей RSA vs ECC
5. RSA подпись (учебная версия)

**Используемые библиотеки:** `pycryptodome`, стандартная библиотека Python

In [ ]:
# Импорты
from math import gcd, isqrt
import time

print("Стандартная библиотека загружена")

---
## Часть 1: Учебная реализация RSA

Реализуем RSA "от нуля" с маленькими простыми числами для полного понимания.

In [ ]:
# Шаг 1: Выбираем простые числа
p = 61
q = 53

# Шаг 2: Вычисляем модуль n
n = p * q
print(f"p = {p}")
print(f"q = {q}")
print(f"n = p * q = {p} * {q} = {n}")

In [ ]:
# Шаг 3: Функция Эйлера phi(n)
phi = (p - 1) * (q - 1)
print(f"phi(n) = (p-1)(q-1) = {p-1} * {q-1} = {phi}")

# Шаг 4: Выбираем открытую экспоненту e
e = 17  # Маленькое e для наглядности (в продакшене обычно 65537)
assert gcd(e, phi) == 1, f"e={e} не взаимно просто с phi={phi}"
print(f"e = {e}, gcd({e}, {phi}) = {gcd(e, phi)} (взаимно просты)")

# Шаг 5: Вычисляем секретную экспоненту d
d = pow(e, -1, phi)  # d = e^(-1) mod phi
print(f"d = e^(-1) mod phi = {d}")
print(f"Проверка: e * d mod phi = {e} * {d} mod {phi} = {(e * d) % phi}")

In [ ]:
# Ключи RSA
public_key = (e, n)
private_key = (d, n)

print(f"Открытый ключ (e, n):  {public_key}")
print(f"Секретный ключ (d, n): {private_key}")

In [ ]:
# Шифрование и дешифрование

def rsa_encrypt(m: int, pub: tuple) -> int:
    """c = m^e mod n"""
    e, n = pub
    return pow(m, e, n)

def rsa_decrypt(c: int, priv: tuple) -> int:
    """m = c^d mod n"""
    d, n = priv
    return pow(c, d, n)

# Тестируем
message = 42
ciphertext = rsa_encrypt(message, public_key)
decrypted = rsa_decrypt(ciphertext, private_key)

print(f"Сообщение:    m = {message}")
print(f"Шифротекст:   c = m^e mod n = {message}^{e} mod {n} = {ciphertext}")
print(f"Дешифрование: m = c^d mod n = {ciphertext}^{d} mod {n} = {decrypted}")
assert decrypted == message, "Ошибка дешифрования!"
print("\nУспех: m == decrypt(encrypt(m))")

In [ ]:
# Шифрование нескольких сообщений
test_messages = [7, 42, 100, 1000, 3000]

print(f"{'m':>6} -> {'c':>6} -> {'m\'':>6}  OK?")
print("-" * 35)
for m in test_messages:
    if m >= n:
        print(f"{m:>6}    ПРОПУСК (m >= n = {n})")
        continue
    c = rsa_encrypt(m, public_key)
    m2 = rsa_decrypt(c, private_key)
    ok = "OK" if m2 == m else "FAIL"
    print(f"{m:>6} -> {c:>6} -> {m2:>6}  {ok}")

In [ ]:
# Полная функция генерации ключей RSA

def generate_rsa_keypair(p: int, q: int, e: int = 65537):
    """Генерация RSA ключей (учебная версия с маленькими числами)."""
    assert p != q, "p и q должны быть разными"
    n = p * q
    phi = (p - 1) * (q - 1)
    
    assert gcd(e, phi) == 1, f"e={e} не взаимно просто с phi={phi}"
    
    d = pow(e, -1, phi)
    
    return {
        'public': (e, n),
        'private': (d, n),
        'p': p, 'q': q,
        'phi': phi,
        'e': e, 'd': d, 'n': n
    }

# Тест с разными простыми числами
keys = generate_rsa_keypair(p=101, q=103, e=17)
print(f"n = {keys['n']}")
print(f"phi = {keys['phi']}")
print(f"e = {keys['e']}, d = {keys['d']}")

# Проверяем
m = 1234
c = rsa_encrypt(m, keys['public'])
m2 = rsa_decrypt(c, keys['private'])
print(f"\n{m} -> {c} -> {m2} ({'OK' if m2 == m else 'FAIL'})")

---
## Часть 2: Продакшен RSA с pycryptodome

Реальный RSA использует ключи >= 2048 бит и паддинг OAEP.

In [ ]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256

# Генерация ключей RSA-2048
key = RSA.generate(2048)
private_pem = key.export_key()
public_pem = key.publickey().export_key()

print(f"Размер приватного ключа PEM: {len(private_pem)} байт")
print(f"Размер публичного ключа PEM: {len(public_pem)} байт")
print(f"Размер модуля n: {key.size_in_bits()} бит")
print(f"\nПубличный ключ (первые 100 символов):")
print(public_pem.decode()[:100] + "...")

In [ ]:
# Шифрование с OAEP (безопасный паддинг)
cipher = PKCS1_OAEP.new(key.publickey())
message = "Приватный ключ кошелька: 0x1a2b3c4d".encode('utf-8')
ciphertext = cipher.encrypt(message)

print(f"Открытый текст:   {message.decode()}")
print(f"Размер сообщения: {len(message)} байт")
print(f"Размер шифротекста: {len(ciphertext)} байт")
print(f"Шифротекст (hex): {ciphertext.hex()[:64]}...")

# Дешифрование
decipher = PKCS1_OAEP.new(key)
decrypted = decipher.decrypt(ciphertext)
print(f"\nРасшифровано: {decrypted.decode()}")
assert decrypted == message
print("Успех!")

In [ ]:
# RSA подпись (продакшен)
message = b"Transfer 1 BTC to Alice"
h = SHA256.new(message)

# Подпись приватным ключом
signature = pkcs1_15.new(key).sign(h)
print(f"Сообщение:  {message.decode()}")
print(f"SHA-256:    {h.hexdigest()[:32]}...")
print(f"Подпись:    {signature.hex()[:64]}...")
print(f"Размер подписи: {len(signature)} байт")

# Верификация публичным ключом
try:
    pkcs1_15.new(key.publickey()).verify(h, signature)
    print("\nПодпись ВЕРНА")
except (ValueError, TypeError):
    print("\nПодпись НЕВЕРНА")

---
## Часть 3: Сравнение размеров ключей RSA vs ECC

In [ ]:
# Сравнение RSA и ECC по размеру ключей и скорости

# RSA-2048
start = time.time()
rsa_key = RSA.generate(2048)
rsa_gen_time = time.time() - start

rsa_pub_size = len(rsa_key.publickey().export_key('DER'))
rsa_priv_size = len(rsa_key.export_key('DER'))

# Для сравнения
print("Сравнение RSA-2048 vs ECC-256:")
print(f"{'':20} {'RSA-2048':>12} {'ECC-256':>12}")
print("-" * 46)
print(f"{'Безопасность':20} {'~112 бит':>12} {'~128 бит':>12}")
print(f"{'Приватный ключ':20} {f'{rsa_priv_size} байт':>12} {'32 байта':>12}")
print(f"{'Публичный ключ':20} {f'{rsa_pub_size} байт':>12} {'33 байта':>12}")
print(f"{'Подпись':20} {'256 байт':>12} {'64 байта':>12}")
print(f"{'Генерация ключа':20} {f'{rsa_gen_time:.3f}с':>12} {'~0.001с':>12}")
print(f"\nECC ключи в {rsa_pub_size // 33}x компактнее — критично для блокчейна,")
print(f"где каждая транзакция содержит публичный ключ и подпись.")

---
## Часть 4: Факторизация маленьких n

In [ ]:
# Простой алгоритм факторизации (trial division)

def factor_trial(n: int) -> tuple:
    """Факторизация перебором делителей. O(sqrt(n))."""
    if n % 2 == 0:
        return (2, n // 2)
    for i in range(3, isqrt(n) + 1, 2):
        if n % i == 0:
            return (i, n // i)
    return (n, 1)  # n простое

# Факторизуем наш учебный n
n_small = 61 * 53  # 3233
start = time.time()
p_found, q_found = factor_trial(n_small)
elapsed = time.time() - start

print(f"n = {n_small}")
print(f"Факторизация: {p_found} x {q_found}")
print(f"Время: {elapsed*1e6:.1f} микросекунд")

# Попробуем числа побольше
test_numbers = [
    61 * 53,           # ~3K (тривиально)
    1009 * 1013,       # ~1M
    10007 * 10009,     # ~100M
    100003 * 100019,   # ~10B
]

print(f"\n{'n':>15} {'p':>8} {'q':>8} {'Время':>12}")
print("-" * 50)
for n_test in test_numbers:
    start = time.time()
    p_f, q_f = factor_trial(n_test)
    elapsed = time.time() - start
    print(f"{n_test:>15} {p_f:>8} {q_f:>8} {elapsed*1e6:>10.1f} мкс")

---
## Упражнения

### Упражнение 1: RSA подпись (учебная версия)

Реализуйте учебную RSA подпись:
- `rsa_sign(m, private_key)` -> подпись s = m^d mod n
- `rsa_verify(m, s, public_key)` -> True если s^e mod n == m

Протестируйте: подпишите сообщение, проверьте подпись, измените сообщение — подпись должна стать невалидной.

In [ ]:
# Упражнение 1: ваш код здесь

def rsa_sign(m: int, priv: tuple) -> int:
    """s = m^d mod n"""
    # Ваш код...
    pass

def rsa_verify(m: int, s: int, pub: tuple) -> bool:
    """Проверить: s^e mod n == m?"""
    # Ваш код...
    pass

# Тест:
# keys = generate_rsa_keypair(p=61, q=53, e=17)
# m = 42
# sig = rsa_sign(m, keys['private'])
# print(f"Подпись: {sig}")
# print(f"Верификация m={m}: {rsa_verify(m, sig, keys['public'])}")
# print(f"Верификация m={m+1}: {rsa_verify(m+1, sig, keys['public'])}")

### Упражнение 2: Замер производительности RSA

Сгенерируйте RSA ключи разных размеров (1024, 2048, 4096 бит) с pycryptodome.
Замерьте время генерации, шифрования и дешифрования для каждого размера.

In [ ]:
# Упражнение 2: ваш код здесь

# for bits in [1024, 2048, 4096]:
#     start = time.time()
#     key = RSA.generate(bits)
#     gen_time = time.time() - start
#     
#     cipher = PKCS1_OAEP.new(key.publickey())
#     msg = b"Test message"
#     
#     start = time.time()
#     ct = cipher.encrypt(msg)
#     enc_time = time.time() - start
#     
#     decipher = PKCS1_OAEP.new(key)
#     start = time.time()
#     pt = decipher.decrypt(ct)
#     dec_time = time.time() - start
#     
#     print(f"RSA-{bits}: gen={gen_time:.3f}s, enc={enc_time*1e3:.1f}ms, dec={dec_time*1e3:.1f}ms")

### Упражнение 3: Факторизация учебного RSA

Дан публичный ключ (e=17, n=3233). Факторизуйте n, вычислите секретный ключ d
и расшифруйте шифротекст c=2201.

In [ ]:
# Упражнение 3: ваш код здесь

e_pub = 17
n_pub = 3233
c_secret = 2201

# Шаг 1: Факторизовать n
# p, q = factor_trial(n_pub)

# Шаг 2: Вычислить phi и d
# phi = ...
# d = ...

# Шаг 3: Расшифровать
# m = pow(c_secret, d, n_pub)
# print(f"Расшифрованное сообщение: {m}")

---
## Челлендж: Атака Винера (Wiener's Attack)

Если секретная экспонента d маленькая (d < n^0.25 / 3), её можно найти через
цепные дроби (continued fractions) разложения e/n.

**Задача:** Реализуйте атаку Винера. Дано e и n (с маленьким d), найдите d.

**Подсказка:** Разложите e/n в цепную дробь. Для каждого подходящего (convergent) k/d
проверьте: если e*d - 1 делится на k, то phi = (e*d - 1)/k. Если phi позволяет
найти p и q (через квадратное уравнение), значит d найден.

In [ ]:
# Челлендж: Wiener's Attack

def continued_fraction(a: int, b: int) -> list:
    """Разложение a/b в цепную дробь."""
    cf = []
    while b:
        q = a // b
        cf.append(q)
        a, b = b, a - q * b
    return cf

def convergents(cf: list):
    """Подходящие дроби (convergents) для цепной дроби."""
    n_prev, n_curr = 0, 1
    d_prev, d_curr = 1, 0
    for q in cf:
        n_prev, n_curr = n_curr, q * n_curr + n_prev
        d_prev, d_curr = d_curr, q * d_curr + d_prev
        yield n_curr, d_curr

def wiener_attack(e: int, n: int):
    """Попытка найти d через атаку Винера."""
    cf = continued_fraction(e, n)
    for k, d in convergents(cf):
        if k == 0:
            continue
        # phi = (e*d - 1) / k
        if (e * d - 1) % k != 0:
            continue
        phi = (e * d - 1) // k
        # p + q = n - phi + 1
        s = n - phi + 1
        # p * q = n
        # p и q — корни x^2 - s*x + n = 0
        discriminant = s * s - 4 * n
        if discriminant < 0:
            continue
        sq = isqrt(discriminant)
        if sq * sq == discriminant:
            p = (s + sq) // 2
            q = (s - sq) // 2
            if p * q == n:
                return d, p, q
    return None

# Тестовый пример с маленьким d
# Для корректного теста нужны p, q где d получается маленьким
p_w = 1009
q_w = 2003
n_w = p_w * q_w
phi_w = (p_w - 1) * (q_w - 1)
d_w = 29  # маленький d
e_w = pow(d_w, -1, phi_w)

print(f"Генерируем уязвимый RSA:")
print(f"n = {n_w}, e = {e_w}")
print(f"Настоящий d = {d_w}")

result = wiener_attack(e_w, n_w)
if result:
    d_found, p_found, q_found = result
    print(f"\nАтака Винера успешна!")
    print(f"Найдено: d = {d_found}, p = {p_found}, q = {q_found}")
    print(f"Правильно: {d_found == d_w}")
else:
    print("Атака не сработала (d слишком большой)")